In [1]:
%matplotlib inline
from pyvista import set_plot_theme, set_jupyter_backend
set_plot_theme('document')
set_jupyter_backend('ipyvtklink')

# XFEAt Edge dislocation

Create an edge dislocation in a bcc Fe crystal and apply a shear stress until the
dislocation becomes mobile.

Author: Alexander Hartmaier  
Institution: ICAMS / Ruhr University Bochum, Germany  
Date: March 2022

This work is licensed under a
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">(CC-BY-NC-SA)</a>

The XFEAt package comes with ABSOLUTELY NO WARRANTY. This is free
software, and you are welcome to redistribute it under the conditions of
the GNU General Public License <a href="http://www.fsf.org/licensing/licenses/gpl.html">(GPLv3)</a>

<img alt="Creative Commons License" style="border-width:0;max-heigt:9px;height:100%;" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png"/></a>
<img alt="GPLv3" style="border-width:0;max-heigt:9px;height:100%;" src="../docs/figures/gplv3-88x31.png"/></a>

In [1]:
import xfeat

# define material of single crystal as dictionary
mat = {
       'name' : 'iron',
       'cs'   : 'bcc',  # crystal structure
       'lp'   : 2.8553,  # lattice parameter in Angstrom
       'mass' : 55.845,  # atomic mass in u
       # define anisotropic elastic constants in GPa
       'C11'  : 243.4,
       'C12'  : 145.0,
       'C44'  : 116.0,
       # W-Bop 6
       # C11. C12, C44 = 2.837825, 1.5317, 0.6256225
       # define crystallograhic orientation of crystal
       'ori_x' : [ 1, 1,  1],
       'ori_y' : [-1, 0,  1],
       'ori_z' : [-1, 2, -1],
       }

# create XFEM model 
mod = xfeat.Model(mat, size=300, verbose=False)
mod.atoms([15, 17, 2])
mod.mesh()
mod.init_dislo([1, 0, 0])
mod.plot('ubcx')


Succesfully created structure with 6120 atoms.
Generated mesh with 26790 elements and 36632 nodes around atomistic core.
XFEM solution obtained in 35.869s.
Relaxation of atoms obtained in 35.896s.

Created dislocation with Burgers vector ([1. 0. 0.]) in model.
Norm of Burgers vector is 2.472772 A


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# iterate into relaxed configuration
for i in range(1):
    # relax atomic region under shear strain
    mod.atom_bc()  # apply relaxed atom positions as BC to XFEM 
    mod.solve()  # colculate nodal displacements for mechanical equilibrium
    mod.shift_atoms()  # move boundary atoms according to strain field
    mod.relax_atoms(i)  # relax atomic structure with fixed boundary atoms
    mod.plot('uy')